In [351]:
# Libreria importantes para la Tarea
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Import necessary modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score
from sklearn.metrics import classification_report,confusion_matrix

# Se corre una vez para mezclar el dataset y se usa el mezclado
dataSet = pd.read_csv("diabetes.csv")



dataSet = pd.read_csv("diabetes.csv")

selected_columns = ['Glucose','BloodPressure', 'SkinThickness', 'Insulin','BMI']  # Add your desired column names

for col in selected_columns:
    # Calculate the mean of non-zero values in the column
    mean_val = dataSet.loc[dataSet[col] != 0, col].mean()
   

    # Replace 0 values with the mean
    dataSet[col] = dataSet[col].replace(0, mean_val)


target = ['Outcome'] 
features = list(set(list(dataSet.columns))-set(target))



dataSet.to_csv("datasetSinCeros.csv", index=False)

dataSeta = pd.read_csv("datasetSinCeros.csv")



scaler = MinMaxScaler()

features_to_normalize = dataSeta.columns.difference(["Outcome"])

datos_normalizados=pd.DataFrame(scaler.fit_transform(dataSeta[features_to_normalize]), columns=features_to_normalize)

datos_normalizados["Outcome"]=dataSeta["Outcome"]

datos_normalizados.to_csv("datasetSinCerosN.csv", index=False)



In [344]:
datos_normalizados.describe()

,Age,BMI,BloodPressure,DiabetesPedigreeFunction,Glucose,Insulin,Pregnancies,SkinThickness,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,0.204015,0.291564,0.493930,0.168179,0.501205,0.170130,0.226180,0.240798,0.348958
std,0.196004,0.140596,0.123432,0.141473,0.196361,0.102189,0.198210,0.095554,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.050000,0.190184,0.408163,0.070773,0.359677,0.129207,0.058824,0.195652,0.000000
50%,0.133333,0.290389,0.491863,0.125747,0.470968,0.170130,0.176471,0.240798,0.000000
75%,0.333333,0.376278,0.571429,0.234095,0.620968,0.170130,0.352941,0.271739,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [539]:



dataTraining, dataTesting = train_test_split(datos_normalizados, test_size=0.2, random_state=42)




x_train = dataTraining[features].values
y_train = dataTraining[target].values



x_test = dataTesting[features].values
y_test = dataTesting[target].values


## el default del alpha es 0.0001
mlp = MLPClassifier(hidden_layer_sizes=(8,8), activation='relu', solver='adam', max_iter=500)
# Se entreba con el vector de features y el vector target que son 80% de los datos
mlp.fit(x_train,y_train)

predict_train = mlp.predict(x_train)
predict_test = mlp.predict(x_test)



print(confusion_matrix(y_test,predict_test))
print(classification_report(y_test,predict_test))


c:\Users\tian_\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1098: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[82 17]
 [18 37]]
              precision    recall  f1-score   support

           0       0.82      0.83      0.82        99
           1       0.69      0.67      0.68        55

    accuracy                           0.77       154
   macro avg       0.75      0.75      0.75       154
weighted avg       0.77      0.77      0.77       154

